In [1]:
#!pip3 install pymystem3
import pandas as pd
from pymystem3 import Mystem
from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

xls = pd.ExcelFile('for_hack_2.xlsx')
df_data = pd.read_excel(xls, sheet_name="data")

In [2]:
#не будем использовать для предсказания

first_drop = df_data[(df_data['Вид экстремизма'].isnull())|(df_data['Вид экстремизма']=='???')
       |(df_data['Вид экстремизма']=='Все сразу')|(df_data['Вид экстремизма']=='not identified')]
first_drop

,№№,№,original,item_as_in_the_list,item_type,item_genus,genre,object_form,object_genus,autor,...,storage_medium,Вид экстремизма,Узкая оценка,Узкая оценка-2,Доп. подробности,Сомнительность,url,court,date_of_design,add_date
368,369,315.0,"Листовка «ВОЗВАНИЕ ПРИЗЫВ», обнаруженная 11.04...",Листовка,Листовка,Листовка,Публицистика,текст,бумажный документ,NaN,...,Бумага,???,???,NaN,найдена в подъездах жилых домов №5 по ул. Киро...,0.0,NaN,Советский районный суд Владивостока,2008-10-24 00:00:00,2009-02-05
474,475,402.0,Интернет-ресурс http://reinform.livejournal.com,Интернет-ресурс,Блог на платформе,Веб-сайт,Публицистика,мультимедиа,веб-сайт,NaN,...,Удаленный сервер,???,???,NaN,NaN,0.0,reinform.livejournal.com,Кировский районный суд Уфы,2009-06-01 00:00:00,2009-07-28
566,567,474.0,"Публикация «Слово о свободе», размещенная на о...",Публикация,Статья на веб-сайте,Статья,Публицистика,текст,отдельная веб-страница,NaN,...,Удаленный сервер,???,???,NaN,NaN,0.0,Livejornal.com,Центральный районный суд Кемерово,2009-10-15 00:00:00,2010-01-20
674,675,577.0,"Публикация «Курицы вы, больше никто», размещен...",Публикация,Статья на веб-сайте,Статья,Публицистика,текст,отдельная веб-страница,NaN,...,Удаленный сервер,???,???,NaN,NaN,0.0,ne ukaz,Улетовский районный суд Читинской области,2009-03-23 00:00:00,2010-04-02
1085,1086,683.0,"Газета «Свободная Республика» № 40 (189), соде...","Газета, содержащая статью",Статья в газете/журнале,Статья,Публицистика,текст,периодическое издание,NaN,...,Бумага,???,???,NaN,NaN,0.0,NaN,Советский районный суд Махачкалы,2010-04-21 00:00:00,2010-07-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8416,8417,5307.0,Видеоматериал под названием: «СУРКОВ ОБЕЩАЛ ДО...,Видеофайл,Видеоролик,Видеоролик,NaN,видео,файл,NaN,...,Удаленный сервер,not identified,NaN,NaN,NaN,0.0,ne ukaz,Красногорский городской суд Московской области,18.07.2022,2022-09-20
8420,8421,5311.0,"Материалы, размещенные в русскоязычном сегмент...",Материалы,Пост в соц.сети,Аккаунт соц.сети/группа,NaN,NaN,файл,Абу Мухаммад,...,Удаленный сервер,not identified,NaN,NaN,NaN,0.0,ne ukaz,Октябрьский районный суд г. Екатеринбурга,2022-04-14 00:00:00,2022-10-19
8426,8427,5317.0,"Материал книги «Ценой свободы», автор Блинов К...",Материал книги,Книга,Непериодическое издание,NaN,текст,непериодическое издание,Блинов К.А (Кирилл Ридик),...,Бумага,not identified,NaN,NaN,NaN,0.0,ne ukaz,Советский районный суд г. Нижнего Новгорода,2022-04-22 00:00:00,2022-11-25
8431,8432,5322.0,"Текстовый файл «Программа М. К. У .» авторов, ...",Текстовый файл,Статья в цифровом формате,Непериодическое издание,Политический текст,текст,файл,"Егор «Маньяк», Warrior of Light",...,Удаленный сервер,not identified,NaN,NaN,NaN,0.0,ne ukaz,Таганский районный суд г. Москвы,2022-07-21 00:00:00,2022-12-12


In [3]:
#унифицируем типы:

df_data.loc[df_data['item_genus'] == "Переодическое издание", 'item_genus'] = 'Периодическое издание'
df_data.loc[df_data['item_genus'] == "видеоролик", 'item_genus'] = "Видеоролик"
df_data.loc[df_data['item_genus'] ==  'Аккаунт соц.сети / группа', 'item_genus'] = 'Аккаунт соц.сети/группа'

#худ. фильм и мультфильм переделаем на видеоролик:

df_data.loc[df_data['item_genus'] ==  'Художественный фильм', 'item_genus'] = 'Видеоролик'
df_data.loc[df_data['item_genus'] ==  'Художественный фильм', 'item_genus'] = 'Мультфильм'

#Аудиокнига поменяем на Аудиозапись

df_data.loc[df_data['item_genus'] ==  'Аудиокнига', 'item_genus'] = 'Аудиозапись'

df_data.loc[[8287, 8289, 8290, 8291, 8311, 8312, 8313], 'item_genus'] = 'Аудиозапись'
df_data.loc[[8335, 8401], 'item_genus'] = 'Литературное произведение'

df_data['date_of_design'] = pd.to_datetime(df_data['date_of_design'])
df_data['add_date'] = pd.to_datetime(df_data['add_date'])
df_data['year'] = df_data['date_of_design'].dt.year

In [4]:
for_model = df_data.drop(first_drop.index)

In [5]:
#лемматизируем для модели описания материалов и выбросим стоп-слова:

m = Mystem(disambiguation=True)

st = pd.read_html('https://github.com/tsekh93/russian_stopwords/blob/main/final_stop.txt')[0][1].reset_index()
stopwords = list(st[1])

def lemmatizer(text):
    try:
        lemmas = []
        for word_info in m.analyze(text):
            if 'analysis' in word_info and len(word_info['analysis']) > 0 and word_info['analysis'][0]['lex'] not in stopwords:
                lemma = word_info['analysis'][0]['lex']
            lemmas.append(lemma)
        return ' '.join(list(set(lemmas)))
    except:
        return text

In [6]:
for_model['lemmatized'] = for_model['original'].progress_apply(lemmatizer)

100%|████████████████████████████████████████████| 8327/8327 [00:20<00:00, 404.97it/s]


In [7]:
#посмотрим, что не лемматизировалось:


for_model[for_model['lemmatized'].isnull()]

,№№,№,original,item_as_in_the_list,item_type,item_genus,genre,object_form,object_genus,autor,...,Узкая оценка,Узкая оценка-2,Доп. подробности,Сомнительность,url,court,date_of_design,add_date,year,lemmatized
5502,5503,3190.0,NaN,Картинка,Постер,Изображение,Агитматериал,изображение,файл,NaN,...,Украинский национализм,Русофобия,NaN,0.0,livedemotivator.ru/9007-novye-demotivatory-rep...,Кунцевский районный суд Москвы,2015-10-22,2015-12-22,2015.0,NaN
5930,5931,3354.0,NaN,Видеозапись,Видеоролик,Видеоролик,Публицистика,видео,файл,NaN,...,Русский национализм,NaN,NaN,0.0,vk.com/id259989129,Новочебоксарский городской суд Чувашской Респу...,2015-12-25,2016-03-23,2015.0,NaN
5931,5932,3354.0,NaN,Видеозапись,Видеоролик,Видеоролик,Публицистика,видео,файл,NaN,...,Русский национализм,NaN,NaN,0.0,vk.com/id259989129,Новочебоксарский городской суд Чувашской Респу...,2015-12-25,2016-03-23,2015.0,NaN
5937,5938,3359.0,NaN,Брошюра,Брошюра,Непериодическое издание,Публицистика,текст,непериодическое издание,NaN,...,Ислам,NaN,NaN,0.0,NaN,Калининский районный суд Тюмени,2015-11-26,2016-03-23,2015.0,NaN


In [8]:
#тоже выкинем их:

second_drop = for_model[for_model['lemmatized'].isnull()]

In [9]:
#итоговая таблица:
for_model_2 = for_model.drop(second_drop.index)

In [10]:
#выкинем колонки с номерами:
for_model_2 = for_model_2.drop(columns=['№№', '№'])

In [11]:
for_model_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8323 entries, 0 to 8443
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   original             8323 non-null   object        
 1   item_as_in_the_list  8322 non-null   object        
 2   item_type            8322 non-null   object        
 3   item_genus           8321 non-null   object        
 4   genre                8293 non-null   object        
 5   object_form          8322 non-null   object        
 6   object_genus         8322 non-null   object        
 7   autor                3101 non-null   object        
 8   title                8323 non-null   object        
 9   ident                5350 non-null   float64       
 10  time                 1168 non-null   object        
 11  Edition              69 non-null     object        
 12  publisher_all        2980 non-null   object        
 13  publishing           1498 non-nul

In [12]:
for_model_2

,original,item_as_in_the_list,item_type,item_genus,genre,object_form,object_genus,autor,title,ident,...,Узкая оценка,Узкая оценка-2,Доп. подробности,Сомнительность,url,court,date_of_design,add_date,year,lemmatized
0,"Музыкальный альбом ""Музыка белых"", автор - Муз...",Музыкальный альбом,Музыкальный альбом,Текст песни,Художественный текст,аудио,произведение,Order,Музыка белых,2.0,...,Русский национализм,Расизм,NaN,0.0,NaN,Первомайский районный суд Омска,2006-11-23,2007-07-14,2006.0,белый автор альбом музыкальный музыка группа
1,"Книга ""Книга единобожия"", автор - Мухаммад ибн...",Книга,Книга,Непериодическое издание,Религиозная литература,текст,непериодическое издание,"аль Ваххаб, Мухаммад ибн Абд",Книга единобожия,1.0,...,Ислам,NaN,В Списке - Мухаммад ибн Сулейман ат-Тамими,0.0,NaN,Савеловский районный суд Москвы,2004-04-02,2007-07-14,2004.0,ат источник книга автор тамими издательский ед...
2,Письма Рады земли Кубанской духовно-родовой де...,Письмо,Статья на веб-сайте,Статья,Публицистика,текст,отдельная веб-страница,Лозинский Н.М.;Герасев В.М.,Письма Рады земли Кубанской духовно-родовой де...,2.0,...,Неоязычество,NaN,Духовно-Родовая Держава Русь,0.0,ne ukaz,Первомайский районный суд Краснодара,2006-03-20,2007-07-14,2006.0,духовно автор русь рад кубанский герасев держа...
3,"Печатные материалы в газете ""Для русских людей...",Печатный материал в газете,Статья в газете/журнале,Статья,Публицистика,текст,периодическое издание,NaN,"статья в газете ""Для русских людей"" №1 (1), ию...",2.0,...,Русский национализм,NaN,NaN,0.0,NaN,Тихвинский городской суд Ленинградской области,2004-05-25,2007-07-14,2004.0,печатный август человек октябрь русский июнь н...
4,"Печатные материалы в газете ""Для русских людей...",Печатный материал в газете,Статья в газете/журнале,Статья,Публицистика,текст,периодическое издание,NaN,"статья в газете ""Для русских людей"" №2 (2), ав...",2.0,...,Русский национализм,NaN,NaN,0.0,NaN,Тихвинский городской суд Ленинградской области,2004-05-25,2007-07-14,2004.0,печатный август человек октябрь русский июнь н...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8439,Видеоролик под названием: «Политолог Никита ма...,Видеоролик,Видеоролик,Видеоролик,NaN,видео,файл,Никита Машук,Политолог Никита машук о гос конфликтах,2.0,...,NaN,NaN,NaN,0.0,ne ukaz,Сургутский городской суд Ханты-Мансийского ав...,2022-08-30,2022-12-29,2022.0,суд рот решение слово вариант заканчиваться се...
8440,"Статья «ТАЙНА КРОВИ СТРАШНЕЕ СПИДА», начинающа...",Статья (публикация),Статья,Статья,Публицистика,текст,переодическое издание,NaN,ТАЙНА КРОВИ СТРАШНЕЕ СПИДА,1.0,...,not identified,NaN,NaN,0.0,ne ukaz,Ленинский районный суд г. Перми,2022-07-12,2022-12-29,2022.0,суд решение слово вид заканчиваться мочь тайна...
8441,"Видеозапись, продолжительностью 43 секунды с н...",Видеоролик,Видеоролик,Видеоролик,NaN,видео,файл,NaN,Маленька дiвчинка рiже русню новий хiт вiд мас...,1.0,...,Русофобия,NaN,NaN,0.0,ne ukaz,Сыктывкарский городской суд Республики Коми,2022-10-27,2022-12-29,2022.0,экран изображать русне суд решение вид нацистс...
8442,Информационный материал - аудиозапись и текст ...,Информационный материал-аудиозапись,Музыкальная композиция,Текст песни,NaN,аудио,файл,Скинхед,Зиг хай,1.0,...,Русский национализм,NaN,NaN,0.0,ne ukaz,Железнодорожный районный суд,2022-10-14,2022-12-29,2022.0,железнодорожный суд доберман решение слово зак...


In [13]:
for_model_2['date_of_design'] = pd.to_datetime(for_model_2['date_of_design'])
for_model_2['add_date'] = pd.to_datetime(for_model_2['add_date'])

In [14]:
# делим на обучающую и тестовую выборки:

X = for_model_2['lemmatized']
y = for_model_2['Вид экстремизма']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4345)

In [15]:
#мешок слов

cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)

In [16]:
#логистическая регрессия

lr = LogisticRegression()
#фитим тренировочные
lr.fit(X_train_cv, y_train)
#трансформируем тестовые:

X_test_cv = cv.transform(X_test)

#предсказания

predictions = lr.predict(X_test_cv)

print(classification_report(y_test, predictions))

                             precision    recall  f1-score   support

Антигосударственные призывы       0.92      0.52      0.67        21
         Национальная рознь       0.91      0.98      0.95      1728
     Пособие по созданию ВУ       1.00      0.50      0.67         8
 Призывы к свержению власти       0.00      0.00      0.00         1
         Пропаганда нацизма       0.00      0.00      0.00         9
          Религиозная рознь       0.95      0.87      0.91       626
           Социальная рознь       0.89      0.45      0.60       104

                   accuracy                           0.92      2497
                  macro avg       0.67      0.48      0.54      2497
               weighted avg       0.92      0.92      0.92      2497



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels wit

In [17]:
# сделаем предсказание для записей без меток:

#лемматизируем тексты в том сабсете:
first_drop = first_drop.drop(columns=['№№', '№', 'Вид экстремизма'])

In [18]:
first_drop['lemmatized'] = first_drop['original'].progress_apply(lemmatizer)

100%|██████████████████████████████████████████████| 117/117 [00:00<00:00, 697.76it/s]


In [19]:
#предсказание:
new = cv.transform(first_drop['lemmatized'])
lr.predict(new)

array(['Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Религиозная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Религиозная рознь', 'Религиозная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'Национальная рознь',
       'Национальная рознь', 'Национальная рознь', 'На

In [20]:
first_drop['predicted'] = lr.predict(new)

In [21]:
first_drop.to_csv('pred.csv', index=False)

In [22]:
first_drop.original.values

array(['Листовка «ВОЗВАНИЕ ПРИЗЫВ», обнаруженная 11.04.2005 в подъездах жилых домов № 5 по ул. Кирова, №№ 12, 17 по ул. Амурской в г. Владивостоке',
       'Интернет-ресурс http://reinform.livejournal.com',
       'Публикация «Слово о свободе», размещенная на официально зарегистрированной Интернет - странице на сайте «www.Livejornal.com»',
       'Публикация «Курицы вы, больше никто», размещенная на сайте в сети Интернет',
       'Газета «Свободная Республика» № 40 (189), содержащую статью «Тяжелые похороны. Президент получил от ворот поворот»',
       'Видеоматериал «KAVKAZ» на Интернет-странице по электронному адресу: http:vkontakte.ru/id28497819',
       'Архив старой версии блога, размещенного в сети Интернет на сайте http://iepifanz.liveiournal.com/',
       'Материалы, размещенные в сети Интернет по электронному адресу http://iepifanz.liveiournal.com/',
       'Информационный материал, содержащийся в листовке «Мы не доверяем этой власти!»',
       'Брошюра «Россия ополчается прот